In [1]:
%load_ext autotime

In [2]:
# append path of magellan to system path
import sys
sys.path.append('/scratch/pradap/python-work/anhaidgroup/magellan')

time: 989 µs


In [3]:
import magellan as mg
import pandas as pd

time: 606 ms


In [4]:
# Load input tables 
A = mg.read_csv_metadata('bdata/walmart_amazon/amazonProds_cleaned.csv', low_memory=False)
B = mg.read_csv_metadata('bdata/walmart_amazon/walmartProds_cleaned.csv', low_memory=False)

time: 61.6 ms


In [5]:
len(A), len(B)

(22074, 2554)

time: 8.98 ms


In [6]:
mg.set_key(A, 'id')
mg.set_key(B, 'id')

True

time: 46.5 ms


In [7]:
# Blocking
# AE on brand
# Overlap blocker on title


time: 587 µs


In [8]:
A.columns

Index([u'id', u'amzid', u'brand', u'modelno', u'category1', u'category2',
       u'title', u'price'],
      dtype='object')

time: 1.77 ms


In [9]:
B.columns

Index([u'id', u'wid', u'brand', u'category', u'title', u'price', u'modelno'], dtype='object')

time: 1.76 ms


In [10]:
ab = mg.AttrEquivalenceBlocker()
C1 = ab.block_tables(A, B, 'brand', 'brand',
                    l_output_attrs=['amzid', 'brand', 'title'], 
                    r_output_attrs=['wid', 'brand', 'title'])

time: 133 ms


In [11]:
C1.head()

,_id,ltable_id,rtable_id,ltable_amzid,ltable_brand,ltable_title,rtable_wid,rtable_brand,rtable_title
0,0,1,106,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10747106,Koss,Koss Stereophones With Flexible Headband Design
1,1,1,1314,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10747030,Koss,Koss Earbud Stereophone
2,2,1,1484,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10760275,Koss,Koss Monaural Headset With Noise Canceling Microphone
3,3,861,106,B000E36R96,Koss,Koss Headphone Carrying Case for KSC75 KSC9 KSC7 KSC17 P14 and P16,10747106,Koss,Koss Stereophones With Flexible Headband Design
4,4,861,1314,B000E36R96,Koss,Koss Headphone Carrying Case for KSC75 KSC9 KSC7 KSC17 P14 and P16,10747030,Koss,Koss Earbud Stereophone


time: 17.9 ms


In [12]:
ob = mg.OverlapBlocker()
C2 = ob.block_tables(A, B, 'title', 'title',
                     l_output_attrs=['amzid', 'brand', 'title'], 
                    r_output_attrs=['wid', 'brand', 'title'])

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 28.3 s



Total time elapsed: 19.503 sec


In [ ]:
# combine blocker output

In [13]:
C = mg.combine_blocker_outputs_via_union([C1, C2])

time: 6.33 s


In [14]:
C.head()

,_id,ltable_id,rtable_id,ltable_amzid,ltable_brand,ltable_title,rtable_wid,rtable_brand,rtable_title
0,0,1,71,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,6497801,Panasonic,Panasonic Stereo Earbud Headphones RP-HV21 Green
1,1,1,106,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10747106,Koss,Koss Stereophones With Flexible Headband Design
2,2,1,294,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10745815,Coby,Coby Deep Base Stereo Headphones CV121
3,3,1,491,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,14312671,Coby,Coby Pink High-Performance Isolation Stereo Earphones CVEM79PNK
4,4,1,515,B00005UPF6,Koss,Koss EQ50 3-Band Stereo Equalizer,10965500,iHome,iHOME Portable Stereo Alarm Clock With iPod Dock Gunmetal


time: 103 ms


In [15]:
# debug blocker

time: 139 ms


In [ ]:
D = mg.debug_blocker(A, B, C)

In [ ]:
mg.view_table(D)

In [ ]:
# Sampling and labeling

In [16]:
C3 = C.query('ltable_title == rtable_title')

time: 801 ms


In [17]:
C3['gold'] = 1

time: 55.7 ms


/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
C4 = C.query('ltable_brand != rtable_brand').sample(200)

time: 1.9 s


In [19]:
C4['gold'] = 0

time: 1.13 ms


In [20]:
mg.copy_properties(C, C3)
mg.copy_properties(C, C4)

True

time: 1.74 ms


In [21]:
R = pd.concat([C3, C4])

time: 3.11 ms


In [22]:
S = mg.combine_blocker_outputs_via_union([C3, C4])

time: 11.7 ms


In [23]:
mg.copy_properties(S, R)

True

time: 1.45 ms


In [24]:
R.head()

,_id,ltable_id,rtable_id,ltable_amzid,ltable_brand,ltable_title,rtable_wid,rtable_brand,rtable_title,gold
386096,386096,1241,1283,B004YW7W06,D-Link,D-Link Systems PowerLine AV 4-Port Switch Starter Kit DHP-347AV,16954396,D-Link,D-Link Systems PowerLine AV 4-Port Switch Starter Kit DHP-347AV,1
489210,489210,1617,2267,B003P8WI36,WinCraft,Lyoto Machida Mouse Pad,16642927,Wincraft,Lyoto Machida Mouse Pad,1
897454,897454,2960,185,B004UDJK6S,Tribeca,Los Angeles Dodgers Pink iPhone 4 Hard Case,16642062,Tribeca,Los Angeles Dodgers Pink iPhone 4 Hard Case,1
948359,948359,3145,2193,B000ES8AWI,Garmin,Garmin Fishfinder 140 with 4.7-Inch Display and Dual-Beam Transducer,4660394,Garmin,Garmin Fishfinder 140 with 4.7-Inch Display and Dual-Beam Transducer,1
1588903,1588903,5268,1219,B0046TIL3O,V7,V7 8GB SD Card Secure Digital High Capacity 8 GB Memory Card,15410649,V7,V7 8GB SD Card Secure Digital High Capacity 8 GB Memory Card,1


time: 17 ms


In [25]:
# train test split

time: 537 µs


In [26]:
train_test = mg.split_train_test(R)

time: 8.6 ms


In [27]:
development = train_test['train']
evaluation = train_test ['test'] 

time: 822 µs


In [28]:
development.head()

,_id,ltable_id,rtable_id,ltable_amzid,ltable_brand,ltable_title,rtable_wid,rtable_brand,rtable_title,gold
3700808,3700808,12452,1998,B003X7TRWE,D-Link,D-Link DGS-1005G 5-Port Gigabit Desktop Switch,15529754,D-Link,D-Link DGS-1005G 5-Port Gigabit Desktop Switch,1
1377546,1377546,4564,955,B00008ZPEA,Cables To Go,Cables To Go - 28013 - Pro Series HD15 M M UXGA Monitor Cable - 15ft,14235439,Adesso,Adesso Pro Mechanical Gaming Keyboard,0
1123212,1123212,3740,403,B004MME76K,Panasonic,Panasonic VIERA TC-P50GT30 50-Inch 1080p 3D Plasma HDTV,14251587,Draper,Draper HiDef Grey ShadowBox Clarion Fixed Frame Screen - 133 diagonal HDTV Format,0
6985227,6985227,21818,2198,B000A0BESC,Kingston,Kingston memory - 1 GB - DDR II KTH-XW4300E 1G,12020338,Peerless,Peerless SmartMount Universal Tilt Mount 23 - 46 Screens,0
781715,781715,2596,292,B0001HAGME,Norazza Inc.,Ape Case Digital or Small SLR Mini DVD Case Accessories Bag AC260,931994,Belkin,Belkin Mini CD Drive Cleaning Kit,0


time: 17.1 ms


In [29]:
# Get features

time: 479 µs


In [30]:
feature_table = mg.get_features_for_matching(A, B)

time: 222 ms


In [31]:
feature_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x7fbd207942a8>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef id_i...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x7fbd3384a7d0>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef id_i...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x7fbd205f2ed8>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef id_i...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x7fbd205f2b90>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef id_i...,True
4,brand_brand_jac_qgm_3_qgm_3,brand,brand,qgm_3,qgm_3,jaccard,<function brand_brand_jac_qgm_3_qgm_3 at 0x7fbd1ff42f50>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True


time: 17.3 ms


In [32]:
# Remove id related features
feature_table = feature_table.ix[4:len(feature_table)]

time: 1.26 ms


In [33]:
feature_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
4,brand_brand_jac_qgm_3_qgm_3,brand,brand,qgm_3,qgm_3,jaccard,<function brand_brand_jac_qgm_3_qgm_3 at 0x7fbd1ff42f50>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True
5,brand_brand_cos_dlm_dc0_dlm_dc0,brand,brand,dlm_dc0,dlm_dc0,cosine,<function brand_brand_cos_dlm_dc0_dlm_dc0 at 0x7fbd1ff42e60>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True
6,brand_brand_jac_dlm_dc0_dlm_dc0,brand,brand,dlm_dc0,dlm_dc0,jaccard,<function brand_brand_jac_dlm_dc0_dlm_dc0 at 0x7fbd2060d050>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True
7,brand_brand_mel,brand,brand,None,None,monge_elkan,<function brand_brand_mel at 0x7fbd2060d2a8>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True
8,brand_brand_lev_dist,brand,brand,None,None,lev_dist,<function brand_brand_lev_dist at 0x7fbd2060d140>,from magellan.feature.simfunctions import *\nfrom magellan.feature.tokenizers import *\ndef bran...,True


time: 17.9 ms


In [34]:
feature_vectors = mg.extract_feature_vecs(development, feature_table=feature_table, attrs_after='gold')

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 289 ms



Total time elapsed: 0.275 sec


In [35]:
feature_vectors

,_id,ltable_id,rtable_id,brand_brand_jac_qgm_3_qgm_3,brand_brand_cos_dlm_dc0_dlm_dc0,brand_brand_jac_dlm_dc0_dlm_dc0,brand_brand_mel,brand_brand_lev_dist,brand_brand_lev_sim,brand_brand_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,gold
3700808,3700808,12452,1998,1.000000,1.000000,1.00,1.000000,0.0,1.000000,6.0,...,0.974074,1.0,0.888889,7.0,7.0,1.0,1.000000,0.0,1.000000,1
1377546,1377546,4564,955,0.047619,0.000000,0.00,0.583333,9.0,0.250000,-3.0,...,0.550000,6.0,0.250000,-1.0,2.0,0.0,0.165999,4.0,0.200000,0
1123212,1123212,3740,403,0.000000,0.000000,0.00,0.425926,8.0,0.111111,-2.0,...,0.600000,8.0,0.200000,-2.0,2.0,NaN,NaN,NaN,NaN,0
6985227,6985227,21818,2198,0.000000,0.000000,0.00,0.416667,8.0,0.000000,0.0,...,0.604762,11.0,0.214286,-6.0,1.0,0.0,0.171859,3.0,0.400000,0
781715,781715,2596,292,0.000000,0.000000,0.00,0.414530,12.0,0.076923,-6.0,...,0.455556,5.0,0.166667,0.0,1.0,0.0,0.386746,3.0,0.400000,0
798479,798479,2656,2536,0.000000,0.000000,0.00,0.317460,11.0,0.083333,-4.0,...,0.561905,6.0,0.142857,-1.0,1.0,0.0,0.002679,4.0,0.200000,0
1786480,1786480,5954,1292,0.000000,0.000000,0.00,0.527778,6.0,0.250000,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.139454,5.0,0.166667,0
1223333,1223333,4052,2117,0.000000,0.000000,0.00,0.495238,9.0,0.100000,-2.0,...,0.000000,11.0,0.153846,-5.0,2.0,0.0,0.026896,5.0,0.166667,0
411512,411512,1334,232,0.000000,0.000000,0.00,0.518519,7.0,0.222222,-1.0,...,0.490741,8.0,0.111111,0.0,2.0,0.0,0.643409,4.0,0.200000,0
1535871,1535871,5088,1074,0.000000,0.000000,0.00,0.464286,6.0,0.142857,-2.0,...,0.451282,13.0,0.000000,-3.0,1.0,0.0,0.744806,5.0,0.166667,0


time: 142 ms


In [55]:
feature_vectors.columns

Index([u'_id', u'ltable_id', u'rtable_id', u'brand_brand_jac_qgm_3_qgm_3',
       u'brand_brand_cos_dlm_dc0_dlm_dc0', u'brand_brand_jac_dlm_dc0_dlm_dc0',
       u'brand_brand_mel', u'brand_brand_lev_dist', u'brand_brand_lev_sim',
       u'brand_brand_nmw', u'brand_brand_sw',
       u'modelno_modelno_jac_qgm_3_qgm_3',
       u'modelno_modelno_cos_dlm_dc0_dlm_dc0',
       u'modelno_modelno_jac_dlm_dc0_dlm_dc0', u'modelno_modelno_mel',
       u'modelno_modelno_lev_dist', u'modelno_modelno_lev_sim',
       u'modelno_modelno_nmw', u'modelno_modelno_sw', u'price_price_exm',
       u'price_price_anm', u'price_price_lev_dist', u'price_price_lev_sim',
       u'gold'],
      dtype='object')

time: 2.22 ms


In [36]:
feature_vectors.fillna(0, inplace=True)

time: 35.3 ms


In [37]:
# Create matchers
dt = mg.DTMatcher(name='DecisitionTree')
rf = mg.RFMatcher(name='RandomForest')
nb = mg.NBMatcher(name='NaiveBayes')
svm = mg.SVMMatcher(name='SVM')
linreg = mg.LinRegMatcher(name='LinReg')
logreg = mg.LogRegMatcher(name='LogReg')


time: 2.42 ms


In [38]:
result = mg.select_matcher([dt, rf, nb, svm, linreg, logreg], table=feature_vectors, 
                           exclude_attrs=['_id', 'ltable_id', 'rtable_id'],
                           target_attr = 'gold')

time: 155 ms


/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisitionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x7fbd20217790>,5,0.75,1.0,1.0,1.0,1.0,0.95
1,RandomForest,<magellan.matcher.rfmatcher.RFMatcher object at 0x7fbd202176d0>,5,1.00,1.0,1.0,1.0,1.0,1.00
2,NaiveBayes,<magellan.matcher.nbmatcher.NBMatcher object at 0x7fbd20217390>,5,1.00,1.0,1.0,1.0,1.0,1.00
3,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x7fbd501d1090>,5,1.00,1.0,0.0,0.0,1.0,0.60
4,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x7fbd20217490>,5,1.00,1.0,1.0,1.0,1.0,1.00
5,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x7fbd20217c10>,5,1.00,1.0,1.0,1.0,1.0,1.00


time: 16.9 ms


In [40]:
train_test = mg.split_train_test(feature_vectors)
train = train_test['train']
test = train_test['test']

time: 7.72 ms


In [41]:
train

,_id,ltable_id,rtable_id,brand_brand_jac_qgm_3_qgm_3,brand_brand_cos_dlm_dc0_dlm_dc0,brand_brand_jac_dlm_dc0_dlm_dc0,brand_brand_mel,brand_brand_lev_dist,brand_brand_lev_sim,brand_brand_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,gold
5069246,5069246,16726,1956,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.497038,4.0,0.200000,0
5449929,5449929,17481,1569,0.045455,0.000000,0.00,0.464773,10.0,0.090909,-2.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0
3618021,3618021,12148,2424,0.000000,0.000000,0.00,0.296296,10.0,0.166667,-1.0,...,0.400673,10.0,0.090909,-1.0,1.0,0.0,0.062461,3.0,0.500000,0
1123212,1123212,3740,403,0.000000,0.000000,0.00,0.425926,8.0,0.111111,-2.0,...,0.600000,8.0,0.200000,-2.0,2.0,0.0,0.000000,0.0,0.000000,0
2568652,2568652,8545,2328,0.095238,0.000000,0.00,0.495000,13.0,0.133333,-9.0,...,1.000000,0.0,1.000000,11.0,11.0,0.0,0.547138,4.0,0.200000,1
5535517,5535517,17663,488,0.000000,0.000000,0.00,0.361111,7.0,0.125000,-1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.012034,5.0,0.285714,0
4506823,4506823,15171,2227,0.000000,0.000000,0.00,0.000000,5.0,0.000000,-2.0,...,0.490079,15.0,0.062500,-2.0,2.0,0.0,0.560861,4.0,0.200000,0
4475025,4475025,15063,1411,0.000000,0.000000,0.00,0.430556,7.0,0.125000,-1.0,...,0.000000,6.0,0.000000,-1.0,0.0,0.0,0.030414,5.0,0.166667,0
697349,697349,2303,401,0.000000,0.000000,0.00,0.412698,13.0,0.071429,-7.0,...,0.000000,40.0,0.000000,-34.0,0.0,0.0,0.001941,6.0,0.000000,0
2063845,2063845,6877,195,0.705882,0.408248,0.25,0.984615,1.0,0.923077,11.0,...,1.000000,0.0,1.000000,6.0,6.0,0.0,0.925898,1.0,0.800000,1


time: 135 ms


In [42]:
feature_vectors.iloc[0, 2]

1998.0

time: 1.9 ms


In [43]:
mg.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable_id', 'rtable_id'],
                           target_attr = 'gold')

time: 10.9 s


In [53]:
G = mg.extract_feature_vecs(evaluation, feature_table=feature_table, attrs_after='gold'
                           )

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 292 ms



Total time elapsed: 0.278 sec


In [54]:
G.fillna(0, inplace=True)

time: 37 ms


In [58]:
dt.fit(table=feature_vectors, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='gold')

time: 2.17 ms


In [63]:
predicted = dt.predict(table=G, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], target_attr = 'predictions', append=True, inplace=False
                           )

time: 2.53 ms


In [64]:
predicted

,_id,ltable_id,rtable_id,brand_brand_jac_qgm_3_qgm_3,brand_brand_cos_dlm_dc0_dlm_dc0,brand_brand_jac_dlm_dc0_dlm_dc0,brand_brand_mel,brand_brand_lev_dist,brand_brand_lev_sim,brand_brand_nmw,...,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,gold,predictions
1579294,1579294,5232,979,0.000000,0.0,0.0,0.447619,6.0,0.142857,-1.0,...,6.0,0.142857,0.0,1.0,0.0,0.034336,5.0,0.166667,0,0
2886651,2886651,9650,232,0.000000,0.0,0.0,0.000000,9.0,0.000000,-2.0,...,0.0,0.000000,0.0,0.0,0.0,0.989701,1.0,0.800000,0,0
3126866,3126866,10482,1908,1.000000,1.0,1.0,1.000000,0.0,1.000000,7.0,...,0.0,1.000000,16.0,16.0,0.0,0.572472,3.0,0.400000,1,1
478758,478758,1578,1298,0.000000,0.0,0.0,0.000000,5.0,0.000000,-3.0,...,12.0,0.142857,-1.0,2.0,0.0,0.139413,3.0,0.400000,0,0
647947,647947,2134,1045,0.000000,0.0,0.0,0.472222,5.0,0.166667,-1.0,...,12.0,0.076923,-5.0,1.0,0.0,0.272760,3.0,0.500000,0,0
1796497,1796497,5986,636,0.000000,0.0,0.0,0.000000,8.0,0.000000,-3.0,...,6.0,0.142857,-1.0,1.0,0.0,0.148547,5.0,0.166667,0,0
6964482,6964482,21758,926,0.071429,0.0,0.0,0.537500,7.0,0.125000,-4.0,...,3.0,0.727273,5.0,7.0,0.0,0.695000,3.0,0.400000,1,1
3859045,3859045,12981,2508,0.000000,0.0,0.0,0.577778,5.0,0.166667,0.0,...,13.0,0.071429,-6.0,1.0,0.0,0.856928,3.0,0.400000,0,0
897454,897454,2960,185,1.000000,1.0,1.0,1.000000,0.0,1.000000,7.0,...,0.0,0.000000,0.0,0.0,1.0,1.000000,0.0,1.000000,1,1
2630511,2630511,8740,837,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,1.000000,12.0,12.0,0.0,0.218040,2.0,0.600000,1,1


time: 145 ms


In [67]:
mg.print_eval_summary(mg.eval_matches(predicted, 'gold', 'predictions'))

Precision : 90.0% (18/20)
Recall : 100.0% (18/18)
F1 : 94.74%
False positives : 2 (out of 20 positive predictions)
False negatives : 0 (out of 98 negative predictions)
time: 8.5 ms
